Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.neighbors import NearestNeighbors

songs= pd.read_csv('../spotify_songs.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


Cargamos el csv de vector caracteristico de las canciones

In [4]:
features = pd.read_csv('../new_features.csv')
features.head()

,track_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [5]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Limpieza de datos nulos y duplicados

In [6]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Obtenemos el tfidvectorizer de nuestra dataframe

In [7]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<18192x102102 sparse matrix of type '<class 'numpy.float64'>'
	with 2120817 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [8]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
15792                         Feel It Still  Portugal. The Man
17315                         Feel It Still  Portugal. The Man
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
15718           Rebel Rebel - 2014 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [9]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
15792                         Feel It Still  Portugal. The Man
17315                         Feel It Still  Portugal. The Man
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [10]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
9936                              Pumped Up          Klingande
11488                               McQueen          TrifeDrew
12583           Rebel Rebel - 1997 Remaster        David Bowie
14384           Rebel Rebel - 1999 Remaster        David Bowie
15397                             Pumped Up          Klingande
15718           Rebel Rebel - 2014 Remaster        David Bowie
15784              Rebel Rebel - Remastered        David Bowie
15792                         Feel It Still  Portugal. The Man
16983                         Teenage Kicks     The Undertones
17315                         Feel It Still  Portugal. The Man
18304                       Pumped Up Kicks  Foster The People


In [11]:
first_value = songs.loc[0]
print(first_value)

track_id                                               0017A6SJgTbfQVU2EtsPNo
track_name                                                           Pangarap
track_artist                                                  Barbie's Cradle
lyrics                      Minsan pa Nang ako'y napalingon Hindi ko alam ...
track_popularity                                                           41
track_album_id                                         1srJQ0njEQgd8w4XSqI4JQ
track_album_name                                                         Trip
track_album_release_date                                           2001-01-01
playlist_name                                              Pinoy Classic Rock
playlist_id                                            37i9dQZF1DWYDQ8wBxd7xt
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [12]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [13]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(18192, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [14]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,1.131617,-0.007011,0.026169,0.091561,-0.226135,0.142755,...,-0.038700,-0.071268,0.033733,0.286010,0.025300,0.135885,-0.211669,-0.454100,-0.269868,0.337329
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.045167,0.125351,0.270290,-0.034200,-0.112064,0.084561,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.039950,0.185921,-0.557007
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.364316,0.343706,-0.195609,0.008802,0.452232,-0.079514,...,0.190457,0.202202,0.133856,-0.452725,-0.338745,-0.050033,0.028329,0.140822,0.259863,-1.065271
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.040475,-0.138258,-0.016978,0.032210,0.061145,-0.143123,...,0.121772,-0.302128,-0.131246,-0.228789,-0.362044,-0.027669,0.188981,0.245499,-0.523468,0.374002
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.330823,0.310332,-0.003732,-0.220356,-0.297301,0.333718,...,-0.667551,0.135213,0.413494,-0.125062,-0.056369,0.409041,-0.067511,0.096366,-0.513577,-1.521303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15601,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15602,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15603,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [15]:
combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
combined_data = combined_data.drop_duplicates() #Elimino los duplicados

combined_data.shape

(15605, 364)

Ahora realizaremos la normalizacion del vector caracteristico para eso de la dataframe mezclada solo eligiremos columnas pertenecientes al vector caracteristico:
feature1 hasta feature50

In [16]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,1.131617,-0.007011,0.026169,0.091561,-0.226135,0.142755,-0.118433,-0.099043,-0.168421,0.134076,...,-0.038700,-0.071268,0.033733,0.286010,0.025300,0.135885,-0.211669,-0.454100,-0.269868,0.337329
1,0.045167,0.125351,0.270290,-0.034200,-0.112064,0.084561,0.030843,0.466074,0.163689,0.139688,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.039950,0.185921,-0.557007
2,0.364316,0.343706,-0.195609,0.008802,0.452232,-0.079514,-0.167727,-0.010781,-0.022688,-0.129539,...,0.190457,0.202202,0.133856,-0.452725,-0.338745,-0.050033,0.028329,0.140822,0.259863,-1.065271
3,0.040475,-0.138258,-0.016978,0.032210,0.061145,-0.143123,0.369927,-0.110431,0.088467,-0.037289,...,0.121772,-0.302128,-0.131246,-0.228789,-0.362044,-0.027669,0.188981,0.245499,-0.523468,0.374002
4,-0.330823,0.310332,-0.003732,-0.220356,-0.297301,0.333718,0.013620,0.194298,0.541525,-0.228518,...,-0.667551,0.135213,0.413494,-0.125062,-0.056369,0.409041,-0.067511,0.096366,-0.513577,-1.521303


Alineamos las filas con el conjunto de datos

In [17]:
features_to_normalize = features_to_normalize.loc[combined_data.index]
features_to_normalize



,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,1.131617,-0.007011,0.026169,0.091561,-0.226135,0.142755,-0.118433,-0.099043,-0.168421,0.134076,...,-0.038700,-0.071268,0.033733,0.286010,0.025300,0.135885,-0.211669,-0.454100,-0.269868,0.337329
1,0.045167,0.125351,0.270290,-0.034200,-0.112064,0.084561,0.030843,0.466074,0.163689,0.139688,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.039950,0.185921,-0.557007
2,0.364316,0.343706,-0.195609,0.008802,0.452232,-0.079514,-0.167727,-0.010781,-0.022688,-0.129539,...,0.190457,0.202202,0.133856,-0.452725,-0.338745,-0.050033,0.028329,0.140822,0.259863,-1.065271
3,0.040475,-0.138258,-0.016978,0.032210,0.061145,-0.143123,0.369927,-0.110431,0.088467,-0.037289,...,0.121772,-0.302128,-0.131246,-0.228789,-0.362044,-0.027669,0.188981,0.245499,-0.523468,0.374002
4,-0.330823,0.310332,-0.003732,-0.220356,-0.297301,0.333718,0.013620,0.194298,0.541525,-0.228518,...,-0.667551,0.135213,0.413494,-0.125062,-0.056369,0.409041,-0.067511,0.096366,-0.513577,-1.521303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,-0.602929,-0.277219,-0.493499,0.094192,0.304649,-0.014173,0.122051,0.127419,0.100785,0.084170,...,-0.193526,-0.226324,0.042981,0.040540,0.163899,0.242983,0.437682,0.234670,-0.081395,-1.041992
15601,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,0.119228,0.109646,0.109759,-0.181768,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
15602,0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,-0.241767,-0.009052,-0.114615,-0.296820,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
15603,0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,-0.093999,-0.114929,0.042168,-0.085458,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Normalizamos las características numéricas del vector caracteristico con scaler.

In [18]:
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(features_to_normalize)
scaled_features

array([[ 2.42079989,  0.17590883,  0.22107331, ..., -1.49213977,
        -0.67088242,  0.72553459],
       [ 0.30819814,  0.49835853,  0.93226323, ...,  0.01009477,
         0.44072547, -0.72456814],
       [ 0.92878221,  1.03030115, -0.42502979, ...,  0.31681246,
         0.62105809, -1.54868272],
       ...,
       [ 1.49729465,  1.25795062,  1.6780924 , ..., -0.70953691,
        -0.64343126,  0.58727966],
       [ 1.94015424,  2.52541918,  0.92136865, ..., -0.32006693,
        -0.56473733, -0.59790269],
       [-0.56823133, -0.43233311, -1.06148261, ..., -0.04535605,
        -0.82038151,  0.39414839]])

Combinamos las características normalizadas con las otras columnas en una nueva dataframe con los datos normalizados

In [19]:
normalized_combined_data = pd.concat([combined_data[['track_id', 'track_name', 'track_artist', 'lyrics']], pd.DataFrame(scaled_features, columns=features_to_normalize.columns)], axis=1)
normalized_combined_data.head()


,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,2.420800,0.175909,0.221073,0.402700,-0.731560,0.543706,...,-0.139635,-0.229927,0.091691,0.911141,-0.042196,0.312649,-0.863187,-1.492140,-0.670882,0.725535
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.308198,0.498359,0.932263,-0.010299,-0.338037,0.331046,...,-1.497263,0.296897,-1.415447,1.305770,0.591837,-1.339585,1.600618,0.010095,0.440725,-0.724568
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.928782,1.030301,-0.425030,0.130920,1.608678,-0.268525,...,0.629410,0.692023,0.429759,-1.629808,-1.332075,-0.342474,-0.053804,0.316812,0.621058,-1.548683
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.299073,-0.143828,0.095372,0.207791,0.259501,-0.500969,...,0.398906,-1.008225,-0.465366,-0.859560,-1.414628,-0.263672,0.487989,0.635100,-1.289377,0.784997
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.422914,0.948998,0.133962,-0.621638,-0.977070,1.241535,...,-2.250035,0.466183,1.373968,-0.502780,-0.331565,1.275179,-0.377023,0.181637,-1.265254,-2.288106


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [20]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, f30, f31, f32, f33, f34, f35, f36, f37, f38, f39, f40, f41, f42, f43, f44, f45, f46, f47, f48, f49, f50, f51, f52, f53, f54, f55, f56, f57, f58, f59, f60, f61, f62, f63, f64, f65, f66, f67, f68, f69, f70, f71, f72, f73, f74, f75, f76, f77, f78, f79, f80, f81, f82, f83, f84, f85, f86, f87, f88, f89, f90, f91, f92, f93, f94, f95, f96, ...]
Index: []

[0 rows x 364 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [21]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,2.420800,0.175909,0.221073,0.402700,-0.731560,0.543706,...,-0.139635,-0.229927,0.091691,0.911141,-0.042196,0.312649,-0.863187,-1.492140,-0.670882,0.725535
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.308198,0.498359,0.932263,-0.010299,-0.338037,0.331046,...,-1.497263,0.296897,-1.415447,1.305770,0.591837,-1.339585,1.600618,0.010095,0.440725,-0.724568
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.928782,1.030301,-0.425030,0.130920,1.608678,-0.268525,...,0.629410,0.692023,0.429759,-1.629808,-1.332075,-0.342474,-0.053804,0.316812,0.621058,-1.548683
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.299073,-0.143828,0.095372,0.207791,0.259501,-0.500969,...,0.398906,-1.008225,-0.465366,-0.859560,-1.414628,-0.263672,0.487989,0.635100,-1.289377,0.784997
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.422914,0.948998,0.133962,-0.621638,-0.977070,1.241535,...,-2.250035,0.466183,1.373968,-0.502780,-0.331565,1.275179,-0.377023,0.181637,-1.265254,-2.288106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.952025,-0.482355,-1.292866,0.411340,1.099544,-0.029752,...,-0.659224,-0.752665,0.122919,0.066823,0.448886,0.690035,1.326725,0.602172,-0.211222,-1.510938
15601,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.657811,-0.377240,-0.319221,-0.427428,-0.282365,-0.166754,...,-0.410277,-0.461388,-0.943233,-1.401135,0.641981,0.804716,1.659534,0.668645,0.961055,-0.942646
15602,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",1.497295,1.257951,1.678092,0.752766,0.079474,-0.996244,...,-0.032671,-0.683351,0.016056,0.251010,0.569123,0.408235,0.182935,-0.709537,-0.643431,0.587280
15603,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",1.940154,2.525419,0.921369,-1.223454,-1.369934,0.783441,...,0.087345,0.232761,1.095888,1.103702,-0.075879,-0.916815,-0.676507,-0.320067,-0.564737,-0.597903


Esta parte parece ser innecesaria, pero se realiza y es que a partir del vector caracteristico se normalizara ahora con la columna lyrycs de la cancion para obetener un vector caracteristico mas exacto.

In [22]:
# Seleccionar las características numéricas normalizadas
numeric_features = normalized_combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
numeric_features

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,2.420800,0.175909,0.221073,0.402700,-0.731560,0.543706,-0.441288,-0.403329,-0.691939,0.411068,...,-0.139635,-0.229927,0.091691,0.911141,-0.042196,0.312649,-0.863187,-1.492140,-0.670882,0.725535
1,0.308198,0.498359,0.932263,-0.010299,-0.338037,0.331046,0.109419,1.721289,0.563042,0.432152,...,-1.497263,0.296897,-1.415447,1.305770,0.591837,-1.339585,1.600618,0.010095,0.440725,-0.724568
2,0.928782,1.030301,-0.425030,0.130920,1.608678,-0.268525,-0.623142,-0.071499,-0.141241,-0.579305,...,0.629410,0.692023,0.429759,-1.629808,-1.332075,-0.342474,-0.053804,0.316812,0.621058,-1.548683
3,0.299073,-0.143828,0.095372,0.207791,0.259501,-0.500969,1.360358,-0.446146,0.278792,-0.232733,...,0.398906,-1.008225,-0.465366,-0.859560,-1.414628,-0.263672,0.487989,0.635100,-1.289377,0.784997
4,-0.422914,0.948998,0.133962,-0.621638,-0.977070,1.241535,0.045877,0.699518,1.990811,-0.951158,...,-2.250035,0.466183,1.373968,-0.502780,-0.331565,1.275179,-0.377023,0.181637,-1.265254,-2.288106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,-0.952025,-0.482355,-1.292866,0.411340,1.099544,-0.029752,0.445902,0.448077,0.325341,0.223575,...,-0.659224,-0.752665,0.122919,0.066823,0.448886,0.690035,1.326725,0.602172,-0.211222,-1.510938
15601,-0.657811,-0.377240,-0.319221,-0.427428,-0.282365,-0.166754,0.435484,0.381260,0.359251,-0.775523,...,-0.410277,-0.461388,-0.943233,-1.401135,0.641981,0.804716,1.659534,0.668645,0.961055,-0.942646
15602,1.497295,1.257951,1.678092,0.752766,0.079474,-0.996244,-0.896289,-0.064999,-0.488613,-1.207760,...,-0.032671,-0.683351,0.016056,0.251010,0.569123,0.408235,0.182935,-0.709537,-0.643431,0.587280
15603,1.940154,2.525419,0.921369,-1.223454,-1.369934,0.783441,-0.351149,-0.463055,0.103838,-0.413698,...,0.087345,0.232761,1.095888,1.103702,-0.075879,-0.916815,-0.676507,-0.320067,-0.564737,-0.597903


In [23]:
# Crear un nuevo vectorizador para las letras
text_vectorizer = TfidfVectorizer()
text_features = text_vectorizer.fit_transform(normalized_combined_data["lyrics"])
text_features

<15605x94436 sparse matrix of type '<class 'numpy.float64'>'
	with 1816552 stored elements in Compressed Sparse Row format>

In [24]:
scaler = preprocessing.StandardScaler()
scaled_numeric_features = scaler.fit_transform(numeric_features)
scaled_numeric_features

array([[ 2.42079989,  0.17590883,  0.22107331, ..., -1.49213977,
        -0.67088242,  0.72553459],
       [ 0.30819814,  0.49835853,  0.93226323, ...,  0.01009477,
         0.44072547, -0.72456814],
       [ 0.92878221,  1.03030115, -0.42502979, ...,  0.31681246,
         0.62105809, -1.54868272],
       ...,
       [ 1.49729465,  1.25795062,  1.6780924 , ..., -0.70953691,
        -0.64343126,  0.58727966],
       [ 1.94015424,  2.52541918,  0.92136865, ..., -0.32006693,
        -0.56473733, -0.59790269],
       [-0.56823133, -0.43233311, -1.06148261, ..., -0.04535605,
        -0.82038151,  0.39414839]])

In [25]:
# Combinar las características normalizadas con las otras columnas
combined_features = np.hstack((scaled_numeric_features, text_features.toarray()))
combined_features

array([[ 2.42079989,  0.17590883,  0.22107331, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.30819814,  0.49835853,  0.93226323, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.92878221,  1.03030115, -0.42502979, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.49729465,  1.25795062,  1.6780924 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.94015424,  2.52541918,  0.92136865, ...,  0.        ,
         0.        ,  0.        ],
       [-0.56823133, -0.43233311, -1.06148261, ...,  0.        ,
         0.        ,  0.        ]])

Obtenemos el mismo vector caracteristico no hay diferencia.

In [26]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,2.420800,0.175909,0.221073,0.402700,-0.731560,0.543706,...,-0.139635,-0.229927,0.091691,0.911141,-0.042196,0.312649,-0.863187,-1.492140,-0.670882,0.725535
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.308198,0.498359,0.932263,-0.010299,-0.338037,0.331046,...,-1.497263,0.296897,-1.415447,1.305770,0.591837,-1.339585,1.600618,0.010095,0.440725,-0.724568
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.928782,1.030301,-0.425030,0.130920,1.608678,-0.268525,...,0.629410,0.692023,0.429759,-1.629808,-1.332075,-0.342474,-0.053804,0.316812,0.621058,-1.548683
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.299073,-0.143828,0.095372,0.207791,0.259501,-0.500969,...,0.398906,-1.008225,-0.465366,-0.859560,-1.414628,-0.263672,0.487989,0.635100,-1.289377,0.784997
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.422914,0.948998,0.133962,-0.621638,-0.977070,1.241535,...,-2.250035,0.466183,1.373968,-0.502780,-0.331565,1.275179,-0.377023,0.181637,-1.265254,-2.288106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,7k4t7uLgtOxPwTpFmtJNTY,Tusa,KAROL G,NA ¿Qué pasa contigo? Dímelo Rrr!O-O-Ovy on th...,-0.952025,-0.482355,-1.292866,0.411340,1.099544,-0.029752,...,-0.659224,-0.752665,0.122919,0.066823,0.448886,0.690035,1.326725,0.602172,-0.211222,-1.510938
15601,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.657811,-0.377240,-0.319221,-0.427428,-0.282365,-0.166754,...,-0.410277,-0.461388,-0.943233,-1.401135,0.641981,0.804716,1.659534,0.668645,0.961055,-0.942646
15602,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",1.497295,1.257951,1.678092,0.752766,0.079474,-0.996244,...,-0.032671,-0.683351,0.016056,0.251010,0.569123,0.408235,0.182935,-0.709537,-0.643431,0.587280
15603,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",1.940154,2.525419,0.921369,-1.223454,-1.369934,0.783441,...,0.087345,0.232761,1.095888,1.103702,-0.075879,-0.916815,-0.676507,-0.320067,-0.564737,-0.597903


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

In [28]:
def extract_features(file_path):
    # Cargar el archivo de audio
    y, sr = librosa.load(file_path)

    # Extraer características
    chroma_stft = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)
    print(chroma_stft.shape)
    #Estandarizamos
    chroma_stft = scaler.fit_transform(chroma_stft.T)
    chroma_stft = chroma_stft.T
    # Dividir en segmentos uniformes
    num_segments = 30
    segment_len = chroma_stft.shape[1] // num_segments

    # Tomar la media de cada segmento
    chroma_stft_uniform = np.mean(chroma_stft[:, :num_segments * segment_len].reshape(12, -1, segment_len), axis=2)
    # Concatenar todas las características en un solo vector
    features = chroma_stft_uniform.flatten()
    
    return features

Consulta 2

In [60]:

audio_path="../query2.wav"
scaled_audio_features2=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features2

(12, 103)


,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-0.460937,-1.128964,-1.087556,-1.009121,-0.961926,-0.864108,-0.815424,-0.114463,-0.035641,0.047482,...,-0.279036,-0.831173,0.434079,-1.259053,-0.799501,-0.921848,-1.027043,2.054607,0.70964,0.051815


In [61]:
import numpy as np

valores = "0.04516739,0.12535082,0.2702896,-0.03419992,-0.11206392,0.08456054,0.030843453,0.4660742,0.16368891,0.13968828,0.33374727,-0.054163873,0.02935094,0.34871945,0.1588707,0.2346931,0.23939767,-0.09214655,-0.2545037,0.0005588655,0.022744918,-0.15976188,-0.33167854,-0.11857447,0.000387963,-0.51346445,0.045993116,-0.2017682,-0.16978702,-0.67827487,0.5429606,0.53225833,0.2950049,-0.2897738,-0.46267855,-0.28744328,-0.30939743,-0.17217362,0.41405004,0.37140605,0.22920868,-0.3088844,-0.26783913,-0.18036184,-0.19619434,0.2385127,0.3743453,0.4449715,0.16674282,0.4445032,0.27485454,-0.052748032,-0.16375156,-0.10003064,0.092122436,-0.51136357,-0.1776857,-0.05862747,-0.10928108,-0.74828136,-0.047045026,0.14065695,-0.11897683,-0.5321611,-0.5133654,-0.556486,-0.2564025,-0.4808613,0.39697322,0.9093223,0.6219861,-0.4653923,-0.48106998,-0.36161163,-0.6607821,-0.013315406,0.9493425,1.0046405,0.63953507,0.9241183,0.95958966,0.2690266,-0.5895447,-0.19832903,0.14150703,-0.5799535,-0.0561127,-0.17740051,-0.1253185,-0.7182089,0.19903931,0.38743475,0.18503065,-0.013989761,0.38193613,0.16809887,0.78272027,0.17051089,-0.17399935,-0.1737356,-0.04456736,0.42734748,0.3892661,0.32841772,0.009092406,-0.07088827,-0.14736612,0.060537703,-0.08681487,-0.20688157,-0.3296656,0.19748381,-0.44623944,-0.26228327,0.06360297,-0.53002536,-0.19838929,-0.15654263,-0.22592497,-0.6717705,0.06803573,0.20892192,0.37971494,0.19843715,0.71289694,0.26476723,0.5569012,0.23415464,-0.6523568,-0.71415573,-0.49576932,0.636164,0.43569764,0.27549326,0.1997842,-0.30200467,-0.6308172,-0.64845115,-0.6275204,-0.6855286,-0.8071697,0.19131362,0.05425102,0.051666304,0.12521969,0.51328075,-0.031530283,0.28188592,0.12829448,0.051085893,-0.23350392,-0.13504978,0.27165008,0.014060797,0.7943484,0.27296665,0.6306907,0.24748154,-0.24231191,-0.3956671,-0.082618445,0.7064006,0.450383,0.37179992,0.15082562,0.0041721095,-0.2718756,-0.47499776,-0.40385687,-0.3493754,-0.40987664,0.24923076,-0.23411869,-0.12372799,0.02832978,-0.19701993,-0.12246756,-0.10805938,-0.09964029,-0.32522047,-0.25635344,-0.4710897,0.022542242,-0.3408672,0.08084742,0.017452342,0.15317427,0.14346285,0.57110345,0.29970616,0.42277482,0.06925552,0.016785474,0.042312093,0.0020854957,0.55551314,0.41861066,0.031713333,0.22300285,0.33087808,0.21985112,-0.045959037,-0.44320428,-0.29501534,-0.2323622,-0.44791135,-0.09611098,-0.20032336,-0.19751726,-0.5781016,-0.62616503,-0.63886744,-0.3063515,-0.4862971,-0.12989877,0.03797007,0.12929256,0.18035042,0.5422518,0.60355014,0.25138417,0.032249477,-0.056072496,0.1485147,0.06655782,0.59212726,0.60997486,0.27168992,0.35233608,0.2677654,0.10262944,-0.4169746,-0.4347698,-0.1615559,-0.031246234,-0.3699821,-0.11251178,0.082487844,0.015608589,-0.49803832,-0.57100284,-0.22821654,0.1369299,0.40549734,0.14022303,0.28098974,0.38060752,0.31034636,0.033253722,0.08724102,0.041617762,0.43650645,0.1646747,0.28491375,0.33068618,0.118721895,0.06947497,-0.2930862,-0.042842496,-0.076090895,-0.026822774,-0.4465738,0.0047636414,-0.12646486,0.020300757,-0.52845734,-0.24686502,-0.050269105,-0.10618826,-0.486251,-0.29073256,0.22674005,0.3729478,0.5864361,-0.11577068,0.17690898,-0.30007118,-0.122587904,-0.16813184,-0.1305184,0.05159204,0.06420844,-0.0075858124,-0.20378633,0.17082974,-0.1972063,-0.113964014,-0.52447706,-0.22513606,0.0059922733,-0.1225352,-0.5554532,0.8979928,0.20062234,0.24553512,-0.17664532,-0.24351068,0.046139996,0.09419215,0.37241033,-0.1742375,0.22476006,0.51016706,0.5473243,0.020872915,0.20843947,-0.1435756,0.18843015,-0.18791191,-0.4222567,-0.23408863,0.1304543,0.28933564,0.05952429,0.3365288,-0.08078565,-0.29640785,-0.61206144,-0.54700935,-0.31695533,-0.36896726,-0.33159128,0.2762198,0.5137067,0.33904,-0.30167058,0.07647557,0.15059751,0.25882837,-0.09366818,-0.021809872,0.22324783,0.3712227,0.25662893,0.34376568,0.048727058,0.20808206,0.4504526,-0.21044597,-0.4198053,-0.29233527,0.21820337,0.34563422,0.4215371,0.22287306,0.0077238707,-0.31548578,-0.5094181,-0.577668,-0.4409064,-0.44324216,0.08499923,-0.41262388,0.40074152,0.2042445,-0.3330025,0.518897,0.039949924,0.18592131,-0.55700696"

# Convertir la cadena a una lista de números
valores_lista = [float(valor) for valor in valores.split(',')]

# Convertir la lista a un array numpy
valores_array = np.array(valores_lista)

# Mostrar el array resultante
scaled_audio_features_directed =pd.DataFrame(valores_array).transpose()
scaled_audio_features_directed

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.045167,0.125351,0.27029,-0.0342,-0.112064,0.084561,0.030843,0.466074,0.163689,0.139688,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.03995,0.185921,-0.557007


Consulta 1

In [55]:
"""audio_path="../query.wav"
scaled_audio_features = extract_and_normalize_features(audio_path, scaler, normalized_combined_data)

scaled_audio_features"""
audio_path="../feel.mp3"
scaled_audio_features=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features

(12, 16179)


,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-2.226673,-1.184086,-0.185877,-1.566312,-0.677324,-0.146892,-0.228335,0.153457,0.511744,0.544832,...,0.080548,-0.026478,-0.570141,-0.151215,0.143674,0.490919,0.502081,-0.034166,-0.010151,-0.02791


Consulta 3

In [31]:

"""
audio_path = "../cinderella.mp3"
scaled_audio_features3 = extract_and_normalize_features(audio_path, scaler,normalized_combined_data)

scaled_audio_features3
"""
audio_path="../cinderella.wav"
scaled_audio_features3=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features3

(12, 2960)


,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-1.863582,-0.116671,-0.711491,-0.708828,-0.83005,-0.376432,-0.288283,0.085835,0.40808,0.311751,...,-0.121525,0.550495,0.800007,0.768051,0.75025,0.663379,0.117914,-0.05335,0.643036,-0.100779


EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de dos arrays se logra obtener los k vecinos mas cercanos

# Sin cola de prioridad

In [32]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


# Con cola de prioridad

In [33]:
from queue import PriorityQueue
from sklearn.neighbors import NearestNeighbors

def knn_search_priority_queue(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Initialize the KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extract the numeric features for the KNN model, excluding the first 4 columns
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convert query_audio_features to a NumPy array
    query_features_numeric = query_audio_features.to_numpy()

    # Use the model to find the k nearest neighbors for the audio query
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Create a priority queue to store the k nearest neighbors
    pq = PriorityQueue()

    # Iterate over the indices and distances to add the neighbors to the priority queue
    for i in range(len(indices[0])):
        index = indices[0][i]
        distance = distances[0][i]
        pq.put((distance, index))

    # Get the k nearest neighbors from the priority queue
    nearest_songs = []
    while not pq.empty():
        distance, index = pq.get()
        nearest_songs.append(reference_features.iloc[index, :4])  # Get only the first 4 columns

    return nearest_songs


In [57]:
def knn_search_simple(Q, collection, k):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia para cada fila de la matriz de consulta Q
        distances = [distance(q, C) for q in Q]

        # Obtener el índice del elemento más cercano para cada fila de Q
        indices = np.argsort(distances)

        # Agregar resultados a la lista
        for i in range(len(Q)):
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': distances[i],
                'query_row': i
            })

    result.sort(key=lambda x: x['distance'])  # Ordena por distancia en orden ascendente
    return result[:k+1]  # Devuelve los k vecinos más cercanos




def distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

k_nearest_songs = knn_search_simple(scaled_audio_features_directed.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,7.962185,0
1,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,8.289243,0
2,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,8.479327,0
3,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,9.121811,0
4,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,9.359920,0
5,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,9.541714,0
6,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,9.541714,0
7,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,9.549682,0
8,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin,9.652161,0
9,6RUpJeXT6U4SPEafnr9Wr7,Where The Hood At,DMX,9.839404,0


In [58]:
k_nearest_songs = knn_search_simple(scaled_audio_features.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,9.839712,0
1,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,9.981121,0
2,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,10.022889,0
3,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,10.668727,0
4,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,10.668727,0
5,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,10.894866,0
6,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,10.911745,0
7,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin,10.939061,0
8,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,10.944947,0
9,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,10.944947,0


In [ ]:
import heapq

def knn_search_priority_queue(Q, collection, k):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final
        dist = distance(Q, C)
        heapq.heappush(result, (dist, row[['track_id', 'track_name', 'track_artist']].to_dict()))

    k_nearest_neighbors = heapq.nsmallest(k, result, key=lambda x: x[0])  # Obtener los k vecinos más cercanos
    return [(neighbor[1], neighbor[0]) for neighbor in k_nearest_neighbors]  # Devuelve una lista de tuplas (vecino, distancia)

k_nearest_songs = knn_search_priority_queue(scaled_audio_features3.to_numpy(), normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

,0,1
0,"{'track_id': '5CwOUooch74h0XarhDfAQK', 'track_...",4130.674812
1,"{'track_id': '444KztZ7hPaz6fpZHYOSdD', 'track_...",4130.750035
2,"{'track_id': '6zTABPrCYA8fm5ZpWrzKbz', 'track_...",4130.811477
3,"{'track_id': '5i3qlYyzV6nc4EvCrS1YVY', 'track_...",4130.936234
4,"{'track_id': '2qgudkFpaRq2YcJDWhKW44', 'track_...",4130.942743


## Query 1


In [59]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
3860,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
12733,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."


In [36]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
3798,2gT8OWcAgPg0yawg1qw4Nv,Loma de Cayenas,Vicente Garcia,NA Cuando te vi Llegó a mi vida el alba Quedó ...
7260,3X3P8wgRL2wsLoPtVFO41d,Let's Straighten It Out,Latimore,"Sit yourself down, girl And talk to me And tel..."
15545,7JDyBLjOijEpNJGXn3F7MN,Bossaura - Remastered,Kollegah,"Glocks und Koka, Geld von Hoes aus Osteuropa J..."
13935,6sT92lWDQbnGNuPlEbygaR,In My Hood,50 Cent,Niggas screw they face up at me On some real s...
7442,3ZgQhe1Sv7mnQjpVTJnOFp,"Captain Save a Hoe (feat. The Click, D-Shot, B...",E-40,Uh-iz-I-iz-I-should I save her? I want to be s...


## Query 2

In [37]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1155,1cucFzR8tzyEO1pxgPxYF9,Welcome to Guadalajara,Little el Crack,Humos y coronas en un ritmo chido ¿Quién sale ...
3860,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
12733,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
4687,2t8yVaLvJ0RenpXUIAC52d,a lot,21 Savage,I love you Turn my headphone down a little bit...
8634,4JYUDRtPZuVNi7FAnbHyux,"Flava in Ya Ear Remix (feat. Notorious B.I.G.,...",Craig Mack,"Bad Boys, come out and play You know we had to..."


In [38]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1155,1cucFzR8tzyEO1pxgPxYF9,Welcome to Guadalajara,Little el Crack,Humos y coronas en un ritmo chido ¿Quién sale ...
3860,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
12733,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
4687,2t8yVaLvJ0RenpXUIAC52d,a lot,21 Savage,I love you Turn my headphone down a little bit...
8634,4JYUDRtPZuVNi7FAnbHyux,"Flava in Ya Ear Remix (feat. Notorious B.I.G.,...",Craig Mack,"Bad Boys, come out and play You know we had to..."


## Query 3

In [41]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
6674,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,"NA One time for your mind, one time Yeah, what..."
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
1205,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
6404,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
10177,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,Scarface: As I open my eyes unto a new day I s...
682,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,Only God can judge me (that right?) Only God c...
11507,5sAwKZXy47JNVk03zqWgwI,Top O' the Morning to Ya,House Of Pain,"""She won't come, just when you want it.."" Ya s..."
4493,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,Brothers and sisters Lend me your ears Listen ...


In [42]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
6674,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,"NA One time for your mind, one time Yeah, what..."
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
1205,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
6404,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
10177,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,Scarface: As I open my eyes unto a new day I s...
682,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,Only God can judge me (that right?) Only God c...
11507,5sAwKZXy47JNVk03zqWgwI,Top O' the Morning to Ya,House Of Pain,"""She won't come, just when you want it.."" Ya s..."
4493,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,Brothers and sisters Lend me your ears Listen ...


In [51]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features_directed, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
2736,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,"Uh huh yea, yeah Duro! You gotta let it bump U..."
5034,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."
1205,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
6404,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,"NA ""Money's growin' like grass with the mass a..."
10197,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,Yea Turn the music up a little bit Dah-dah-daa...
10790,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin,Manifest lyrics: Zayvin Lakin Verse: Bitch I a...
15288,7fXb8yG4gCsioqzJNZOv69,Where The Hood At,DMX,"Ayo, y'all niggas must be out your fuckin' min..."


## Range Search
El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier


def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Solo trabajaremos con el vector característico de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Creamos el modelo RadiusNeighborsClassifier
    knn_model = RadiusNeighborsClassifier(radius=radius, metric='euclidean')

    # Entrenamos al modelo con el vector característico de las canciones
    knn_model.fit(reference_features_numeric, reference_features['track_id'])

    # Buscamos vecinos dentro del radio para cada punto de la consulta
    neighbor_indices = knn_model.radius_neighbors(query_features_numeric, return_distance=False)

    # Extraemos información de las canciones encontradas
    song_data = []
    unique_tracks = set()

    for query_index, indices in enumerate(neighbor_indices):
        for neighbor_index in indices:
            track_id = reference_features['track_id'].iloc[neighbor_index]
            if track_id not in unique_tracks:
                unique_tracks.add(track_id)
                song_data.append({
                    'track_id': track_id,
                    'track_name': reference_features['track_name'].iloc[neighbor_index],
                    'track_artist': reference_features['track_artist'].iloc[neighbor_index],
                })

    return song_data

## Query 1
Query 1 normalizada mala diccion y bajo volumen. 

In [ ]:
radius = 21
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result


/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG
1,2CPturRUlpvirYr7VpkXCV,It Ain't Hard to Tell,Nas
2,2gT8OWcAgPg0yawg1qw4Nv,Loma de Cayenas,Vicente Garcia
3,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor
4,3cg9A9GVyZAT2CPevqWr65,Cambio,Ozuna
5,3X3P8wgRL2wsLoPtVFO41d,Let's Straighten It Out,Latimore
6,3ZgQhe1Sv7mnQjpVTJnOFp,"Captain Save a Hoe (feat. The Click, D-Shot, B...",E-40
7,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib
8,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music
9,6sT92lWDQbnGNuPlEbygaR,In My Hood,50 Cent


## Query 2
Query 2 diccion aceptable y volumen aceptable

In [ ]:
radius = 19
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac
1,14ZeVc6AqpEt29RN8IH72x,Wood Wheel,UGK
2,1cucFzR8tzyEO1pxgPxYF9,Welcome to Guadalajara,Little el Crack
3,1oHkXeW88HEtFt6P3TvBIt,Whisper,Blac Youngsta
4,1rME1sbMT1519MjbLJrwc7,My Prerogative,Bobby Brown
5,2g9E9UyZ3pdJ1eqUQW0tfQ,Xxplosive,Dr. Dre
6,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J
7,2pwTzYUTIiwF7Pn8ygXD91,California Love (remix),2Pac
8,2t8yVaLvJ0RenpXUIAC52d,a lot,21 Savage
9,3YDlzGn9svFtz7uPVWThCZ,Don't Be Cruel - Radio Edit,Bobby Brown


## Query 3
La query 3 es mas clara mejor diccion y pronunciacion mejor resultado que la query 1, demostrando que si la consulta es una que no tiene una buena diccion y volumen bajo, obtenemos malos resultados.

In [ ]:
radius = 13
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty
1,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac
2,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr
3,1oHkXeW88HEtFt6P3TvBIt,Whisper,Blac Youngsta
4,1w29UTa5uUvIri2tWtZ12Y,Hip Hop Hooray,Naughty By Nature
5,2hluGQtmPypU3tsQudCQ3U,Tonite,DJ Quik
6,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor
7,39SBJ11H2SKe3Jx3vO3pON,Don't Be Cruel,Bobby Brown
8,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr
9,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas


## Aclaraciones

La data frame tiene una cancion pero de distintas versiones y tiempo, cuenta con detalles que otros no tienen.

In [ ]:
# Filtrar el DataFrame original para obtener todas las instancias de la canción con track_id específico
all_instances = songs[songs['track_id'] == '05xbEV5M8DAQmhFKKlHaDd']

# Contar el número de instancias
num_instances = all_instances.shape[0]

# Imprimir el resultado
print(f"La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite {num_instances} veces.")


La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite 1 veces.


In [ ]:
# Identificar posibles versiones duplicadas basadas en ciertas columnas (por ejemplo, artist_name y title)
possible_duplicates = songs[songs.duplicated(['track_name', 'track_album_name'], keep=False)]

# Mostrar el DataFrame con posibles versiones duplicadas
possible_duplicates


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
10,00hdjyXt6MohKnCyDmhxOL,Una Vaina Loca,Fuego,Fuego Uoh uoh uoh La musiica del futuroo Yeah!...,1,01nV3KuocS1NJHTsJbPkTO,Una Vaina Loca,2011-11-02,MIX LATIN POP°,6IS6XTdbS9qJZgfjNKgpB8,...,-5.589,1,0.0361,0.11400,0.000080,0.0620,0.642,117.009,188213,es
16,00LfFm08VWeZwB0Zlm24AT,Suga Suga,Baby Bash,"So tight, so fly You got me lifted, you got me...",8,2bAkYizbM1rMbOUHpfAV9z,Tha Smokin' Nephew,2003-01-01,90s/00s Hip Hop & RnB,0Ar0Ng9DlAWZtSPBvOQgOa,...,-3.041,0,0.2680,0.68800,0.000008,0.0841,0.535,82.331,239027,en
41,01auWCuHXubb8KPZ0IXF4S,Get Get Down - Radio Edit,Merk & Kremont,"Down, down do-down, now ge-get",37,1P5jv5F4eW3TPh5k10I3Uc,Get Get Down,2015-06-05,Alex Workout,7fRlDx99wBg6i45OCvxwKY,...,-4.087,0,0.0764,0.00724,0.013400,0.4680,0.332,128.032,181875,pl
42,01Bw3xiZ5PQfn0btMWICaQ,Beautiful Life,Lost Frequencies,"Anywhere you are and you'll go, I follow Mount...",30,2WGceQjphDCN6OdXxNBH6x,Beautiful Life,2016-06-03,Tropical Beats,37i9dQZF1DX1bDvyV0IkZ3,...,-6.684,0,0.0376,0.12700,0.034700,0.1390,0.339,119.979,161464,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18382,7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,blink-182,"All the small things True care, truth brings I...",21,5qt11cWjSs5Gbqj2Wyfu38,Enema Of The State,1999-01-01,keg party jukebox,4PjiT7SWnUsTTTvKOuR76f,...,-4.764,1,0.0570,0.01220,0.000000,0.5470,0.695,148.599,168000,tl
18393,7ykaUgkdQWJLsMuOymTV2A,If Not For You,George Harrison,"If not for you Babe, I couldn't even find the ...",59,4I4xtHaIFOzhZfp1NIHkY6,All Things Must Pass (Remastered),1970-11-27,I didn’t know perm stood for permanent (wave),3e6gYPyrTbaB8BWgSHCt5j,...,-6.886,1,0.0317,0.21900,0.003990,0.0820,0.867,116.119,210720,en
18400,7yqU5HODQU91zj8ahUaUX1,I Wanna Dance with Somebody (Who Loves Me),Whitney Houston,Clock strikes upon the hour And the sun begins...,2,1wvEC2yY7koRQYebhrj1ZY,Whitney,1987-06-02,80s Pop & Rock Hits and Album Tracks,0XOIK4m26aeYSD61E5nSVW,...,-8.824,1,0.0453,0.20700,0.000307,0.0888,0.867,118.814,291293,en
18416,7zdoYASH8iYGR1CQV3m8dY,SOS,McClenney,"Hey, hey, oh, oh Hey, hey, oh, oh Hey, hey, oh...",5,7CHix0EuA9xs2RY2sFi5Pv,SOS,2019-02-19,I Love My Neo-Soul,37i9dQZF1DX44dZ4p5QLf4,...,-13.628,0,0.0390,0.57200,0.003940,0.1220,0.555,72.950,187242,en


In [ ]:
# Seleccionar la canción específica (cambiar 'track_name' y 'track_album_name' según tus necesidades)
cancion_seleccionada = songs[(songs['track_name'] == 'Poison')]

# Contar duplicados para la canción seleccionada
duplicados_cancion_seleccionada = cancion_seleccionada.shape[0]

# Imprimir el resultado
print(f"La canción seleccionada se repite {duplicados_cancion_seleccionada} veces.")

cancion_seleccionada


La canción seleccionada se repite 13 veces.


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
237,06yHcjr58IKJnfEK7ko3sD,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",36,3ExqNzkCrQkkknQbrJFqMd,RnB 1990's,2019-07-19,CSR 103:9 (GTA: SA),4sr2xEhXQR5VuZ0LZX8TQ8,...,-10.530,0,0.2100,0.00195,0.004170,0.6330,0.775,111.815,261853,en
622,0dDifktDnbzmGwliFzI0Ld,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,5AR2bbUUDI2XEMarSbs8Cn,Bell Biv DeVoe Greatest Hits (Remastered),2000,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,...,-9.376,0,0.2100,0.00206,0.001830,0.6340,0.783,111.879,261627,en
1705,0RnDu3eYJqbFKz6MHv2ajd,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",48,2QC1IsQIUNdEz0zgWanPkN,20th Century Masters: The Millennium Collectio...,2002-01-01,New Jack Swing - 90s R&B fused w Hip Hop,79xd4wnVuKZK4rJMsL2wPa,...,-6.365,0,0.1920,0.00215,0.002680,0.6270,0.802,111.857,262027,en
3433,1iQpStZpzBb5TOuTDKJKdj,Poison,Alice Cooper,"Your cruel device Your blood, like ice One loo...",59,0ztezUxR9YMkIP1SFRSnV1,Alice Cooper Classicks,1989,80s / Classic Rock,29dTrOurPDrMcrnio2q6hZ,...,-7.786,0,0.1090,0.05150,0.000333,0.1250,0.268,118.631,269600,en
4878,231mi5BkHyJHxQre25S6oS,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,0PPAtp7bE7Itwl0UtG1mXc,20 #1's: 90's R&B,2015-06-15,90's NEW JACK SWING,0dmInkymNnOTWvEFamSNzb,...,-7.312,0,0.2030,0.00188,0.001570,0.5990,0.761,111.871,262053,en
5419,2donM8sTsqYtRpSAwRdPJQ,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",1,7r7Vb7w9Fc3kUDNgIlPIxk,Gold,2005-01-01,90s R&B - The BET Planet Groove/Midnight Love Mix,7LjRWGUdOmBdPG1W1bga4i,...,-7.526,0,0.2280,0.00615,0.005270,0.5030,0.676,111.912,262467,en
7429,39J10NL0mFTAdJbapoo2rC,Poison,Alice Cooper,"Your cruel device Your blood, like ice One loo...",57,7fP5RXmvSPfAY1EgffEPVx,The Best Of Alice Cooper,2007-05-05,Workday: Rock Classics,37i9dQZF1DX1lwxXvReaux,...,-6.096,0,0.1170,0.05240,0.000036,0.1110,0.305,118.794,270787,en
11775,4ZkVOb0mNnm3MFY2LeyqHY,Poison,Martin Garrix,NaN,2,2m4U0aYL6QDgbkL5GFbCmn,Poison,2015-11-02,Jeff Seid Electro House,2Tisn6NZbeofpc57Oc6MGo,...,-6.118,0,0.0363,0.00429,0.769000,0.3770,0.062,128.125,248447,NaN
13185,5NijSs5dAwaIybq1GaRTIe,Poison,Brent Faiyaz,I feel like there's no such thing as a relatio...,68,17dsllSewkPuKbkGWU4chD,Poison,2016-08-03,Urban Contemporary,4Pbs84EQbuAblxlp6Chz0d,...,-10.102,0,0.0650,0.71100,0.000905,0.0859,0.382,117.489,212992,en
